In [1]:
using Revise
using MendelIHT
using SnpArrays
using Random
using GLM
using DelimitedFiles
using Test
using Distributions
using LinearAlgebra

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


# First simulate multivariate Gaussian traits

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n) 

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r);

In [3]:
σ11, σ12, σ22 = sqrt(Σ[1, 1]), sqrt(Σ[1, 2]), sqrt(Σ[2, 2])
true_correlation = σ12 / (σ11 * σ22)
empirical_correlation = cor(Y[:, 1], Y[:, 2])

@show true_correlation
@show empirical_correlation;

LoadError: UndefVarError: Σ not defined

# Run IHT

In [4]:
result = fit_iht(Matrix(Y'), Transpose(xla), Matrix(z'), k=10, max_step=10)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse unknown regression
Link functin = IdentityLink()
Sparsity parameter (k) = 10
Prior weight scaling = off
Doubly sparse projection = off
Debias = off

Converging when tol < 0.0001:
Iteration 1: loglikelihood = -190.2551692133452, tol = 0.7952327339797808
Iteration 2: loglikelihood = 1808.682161396048, tol = 0.13517347084708314
Iteration 3: loglikelihood = 2437.552500878157, tol = 0.04734845208975211
Iteration 4: loglikelihood = 2573.1236645802696, tol = 0.011614347271022793
Iteration 5: loglikelihood = 2599.285603979477, tol = 0.017858824716468415
Iteration 6: logli


IHT estimated 20 nonzero SNP predictors and 0 non-genetic predictors.

Compute time (sec):     3.711843967437744
Final loglikelihood:    2619.469862699537
Iterations:             36

Selected genetic predictors:
20×2 DataFrame
 Row │ Position                 Estimated_β 
     │ Cartesia…                Float64     
─────┼──────────────────────────────────────
   1 │ CartesianIndex(2, 46)      1.73639
   2 │ CartesianIndex(1, 782)    -0.388623
   3 │ CartesianIndex(1, 901)     0.778375
   4 │ CartesianIndex(1, 1204)    0.735256
   5 │ CartesianIndex(1, 1306)   -1.45943
   6 │ CartesianIndex(1, 1655)   -0.182059
   7 │ CartesianIndex(1, 3160)   -0.84684
   8 │ CartesianIndex(1, 4201)    0.31211
   9 │ CartesianIndex(1, 4656)    0.0724518
  10 │ CartesianIndex(2, 4656)    0.0632281
  11 │ CartesianIndex(2, 4797)   -1.19923
  12 │ CartesianIndex(2, 5616)   -0.97377
  13 │ CartesianIndex(1, 6047)    0.150628
  14 │ CartesianIndex(2, 6072)    0.537718
  15 │ CartesianIndex(2, 6573)   -0.592

In [5]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

10×2 Array{Float64,2}:
 -0.388623  -0.402269
  0.778375   0.758756
  0.735256   0.729135
 -1.45943   -1.47163
 -0.182059  -0.172668
 -0.84684   -0.847906
  0.31211    0.296183
  0.0       -0.0034339
  0.150628   0.125965
 -1.22745   -1.24972

In [6]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

10×2 Array{Float64,2}:
  1.73639    1.73729
 -1.19923   -1.19911
  0.0        0.0121193
 -0.97377   -0.969569
  0.537718   0.540525
 -0.592252  -0.609556
  0.506521   0.481189
  0.0       -0.0524866
  0.337317   0.31182
  1.27771    1.29813

In [7]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
 1.17665  1.22512
 1.19901  1.23674
 1.19901  1.23674
 1.46801  1.494